title: Domain Driven Design in Analytics

author: Simone Carolini

date: 2022-07-01

region: EU

tags: analytic engineering - Tooling And Services

summary: Something Something Something Something Something

<div class="alert alert-block alert-info">
    <H1>HTML Creator </H1>

</div>

This script is meant to create HTML files out of the output of the cells of Jupyter Notebooks.

To use this script you just need to add your research file name in the cell below (e.g. '20210420_backfilling_overdraft_decisions'), run the whole script and voila, your HTML file will be ready in the selected folder.

In [ ]:
file_name = "20210420_backfilling_overdraft_decisions"

In [ ]:
if file_name.endswith(".ipynb"):
    file_name = file_name
else:
    file_name = file_name + ".ipynb"

file_name

In [ ]:
!pip install beautifulsoup4
!pip install jupyter_contrib_nbextensions
!jupyter contrib nbextension install --user

In [ ]:
import re
from bs4 import BeautifulSoup
import codecs
import os
import pandas as pd

In [ ]:
# Get directory list of yml files
source_data = []

for r, d, f in os.walk("/app"):
    for file in f:
        if file == file_name:
            source_data.append((d, os.path.join(r, file)))

df = pd.DataFrame(source_data, columns=["folder", "file"])

if len(df.index) > 0:
    dir_path = df.file.iloc[0]  # Only take the first result
else:
    raise SystemExit("No files found for the given file name!")

In [ ]:
dir_path

In [ ]:
html_path = dir_path.replace(".ipynb", ".html")
html_path

In [ ]:
# Create HTML file
!jupyter nbconvert --to html_embed  $dir_path --no-input

# Open file to edit
html = codecs.open(html_path, "r")
# Create soup out of HTML
soup = BeautifulSoup(html)
# create new tags we will be adding to the html,
dr_tag = soup.new_tag("script")
dr_script = """
   $( document ).ready(function() {
        $('div.output_subarea').css('text-align', 'center');
        $('.jp-Cell').css('margin', '0 auto');
        $('.jp-MarkdownOutput').css('margin', '0 auto');
        $('.jp-OutputArea-output pre').css('margin', '0 auto');
        $('.jp-OutputArea-output.jp-RenderedHTMLCommon table').css('margin', '0 auto');
        $('.jp-Cell').css('max-width', '1200px');
        $('.jp-MarkdownOutput').css('max-width', '800px');
        $('.jp-OutputArea-output pre').css('max-width', '800px');
        $('.jp-Cell-outputWrapper').css('text-align', 'center');
        $('.jp-Cell-outputWrapper').css('margin-top', '20');
        $('.jp-Cell-outputWrapper').css('margin-bottom', '20');
        $('body, p, ul, table').css('font-family', 'Montserrat, sans-serif');
        $('.jp-OutputArea-output pre').css('font-family', 'Montserrat, sans-serif');
        $('h1, h2, h3, h4').css({'font-family': 'Karla, sans-serif', 'padding-top': '70px'});
    });
"""

dr_tag.append(dr_script)
jq_tag = soup.new_tag(
    "script", src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"
)
font_link = soup.new_tag("link", href="https://fonts.gstatic.com", rel="preconnect")
montserrat_link = soup.new_tag(
    "link",
    href="https://fonts.googleapis.com/css2?family=Montserrat:wght@300&display=swap",
    rel="stylesheet",
)
karla_link = soup.new_tag(
    "link",
    href="https://fonts.googleapis.com/css2?family=Karla&display=swap",
    rel="stylesheet",
)
tags = [dr_tag, jq_tag, font_link, montserrat_link, karla_link]


# insert created tags\n",
for tag in tags:
    soup.head.insert(1, tag)

# open another file for writing and updating html
with open(html_path, "w") as fp:
    # write the current soup content
    fp.write(soup.prettify())